In [1]:
#%% Import libraries
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import numpy as np

# Neural Network Components
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

2025-11-01 02:01:03.988267: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 02:01:04.135545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-01 02:01:06.045236: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import os
os.chdir(r"/tf/Tarea_6") 
from ANN_creator_module import NeuralNetworkConstructor

In [6]:
def wandb_block(params):
    config_directory = params
    trial_number = config_directory["name"]
    
    run = wandb.init(
        settings=wandb.Settings(x_disable_stats=False, x_stats_sampling_interval = sampling_interval),
        # set the wandb project where this run will be logged
        name = f"Trial_{trial_number}",
        project = config_directory['project'],
        group = config_directory['project'],
        # track hyperparameters and run metadata with wandb.config
        config = config_director
    )
    time.sleep(3.0)
    return run

In [1]:
class ModelBuilder:
    def __init__(self, trial, inputs, total_classes):
        # INITIAL CONFIGURATION
        self.project = 'Plant_Pathology_Classifier'
        self.group = 'Mini_Train'
        self.loss = 'categorical_crossentropy'
        self.metrics = 'val_loss'
        self.batch_size = 25
        self.sampling_interval = 20
        self.inputs=inputs
        self.total_classes = total_classes
        
        # PRIMARY OPTUNA SUGGESTIONS
        self.trial_number = trial
        self.conv_blocks = trial.suggest_int('conv_blocks', 1, 5)
        self.maxpooling_rate = trial.suggest_int('maxpooling_tate', 1, 5)
        self.bottleneck_length = trial.suggest_int('conv_blocks', 1, 5)
        self.total_
        self.filters = trial.suggest_int('conv_blocks', 16, 256, step=16)
        self.complete_length = self.conv_blocks*self.maxpooling_rate
        self.bottleneck_dropout = trial.suggest_float('bottleneck_dropout', 0.1, 0.5)
        self.optimizer = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop", "Nadam"])
        self.classifier_activation = 'softmax'
        
        # INITIALIZING THE CODER LISTS
        self.downwards_activations = [None]*self.complete_length
        self.downwards_dropouts = [None]*self.maxpooling_rate
        self.downwards_regularizers = [None]*self.complete_length
        
        # INITIALIZING THE BOTTLENECK LISTS
        self.bottleneck_activations = [None]*self.bottleneck_length
        self.bottleneck_regularizers = [None]*self.bottleneck_length
        
        # INITIALIZING THE DECODER LISTS
        self.upwards_activations = [None]*self.complete_length
        self.upwards_dropouts = [None]*self.maxpooling_rate 
        self.upwards_regularizers = [None]*self.complete_length
        
        
        
        # SECONDARY OPTUNA SUGGESTIONS
        for i in range(self.bottleneck_length):
            activation_name = trial.suggest_categorical(f'BN_activation{i}', 
                                                        ['relu','tanh','leaky_relu','elu','silu','mish'])
            regularizer_name = trial.suggest_categorical(f'BN_regularizer{i}',
                                                        [None, 'L1L2', "L1", "L2"])
            self.bottleneck_activations[i] = activation_name
            self.bottleneck_regularizers[i] = regularizer_name
        
        for i in range(self.total_length):
            activation_name = trial.suggest_categorical(f'DW_activation{i}', 
                                                        ['relu','tanh','leaky_relu','elu','silu','mish'])
            dropout_value = trial.suggest_float(f'DW_dropout{i}',0.1,0.5)
            regularizer_name = trial.suggest_categorical(f'DW_regularizer{i}',
                                                        [None, 'L1L2', "L1", "L2"])
            self.downwards_activations[i] = activation_name
            self.downwards_dropouts[i] = dropout_value
            self.downwards_regularizers[i] = regularizer_name

        for i in range(self.total_length):
            activation_name = trial.suggest_categorical(f'UW_activation{i}', 
                                                        ['relu','tanh','leaky_relu','elu','silu','mish'])
            dropout_value = trial.suggest_float(f'UW_dropout{i}',0.1,0.5)
            regularizer_name = trial.suggest_categorical(f'UW_regularizer{i}',
                                                        [None, 'L1L2', "L1", "L2"])
            self.upwards_activations[i] = activation_name
            self.upwards_dropouts[i] = dropout_value
            self.upwards_regularizers[i] = regularizer_name
        
        self.model = self._build_model()
    
    def get_params(self, params_type=1):
        wandb_params = {
            'name': self.trial_number,
            'project': self.project,
            'group': self.group,
            'optimizer': self.optimizer,
            'loss': self.loss,
            'batch_size': self.batch_size,
            'filters': self.filters,
        }
        ann_params = {
            'filters': self.filters,
            'conv_blocks': self.conv_blocks, 
            'maxpooling_rate': self.maxpooling_rate,
            'bottleneck_lenght': self.bottleneck_lenght,
            'downwards_activations': self.downwards_activations,
            'downwards_dropouts': self.downwards_dropouts,
            'downwards_regularizers': self.downwards_regularizers,
            'bottleneck_activations': self.bottleneck_activations,
            'bottleneck_dropout': self.bottleneck_dropout,
            'bottleneck_regularizers': self.bottleneck_regularizers,
            'upwards_activations':self.upwards_activations,
            'upwards_regularizers':self.upwards_regularizers,
            'upwards_dropouts':self.upwards_dropouts,
            'classifier_activation': self.classifier_activation,
        }
        if params_type==0:
            params = wandb_params
        elif params_type==1: 
            params = ann_params
        elif params_type==2:
            params = wand_params | ann_params
        else:
            warnings.warn("params_type must be 0 (for wandb parameters), 1 (for ann parameters) or 2 (for all parameters)", 
                          UserWarning)
            params = None
        return params
        
    def _build_model(self):
        inputs = self.inputs
        total_categories = self.total_categories
        params = self.get_params(1)
        Neural_network_constructor = NeuralNetworkConstructor(inputs, total_categories, **params)
        outputs = NeuralNetworkConstructor.complete_outputs()
        model = keras.Model(inputs=inputs,
                            outputs=outputs)
        model.compile(optimizer=self.optimizer,
                     loss=self.loss,
                     metrics=[self.metrics])
        return model
    
    def get_model(self):
        return self.model

In [5]:
def objective(trial):
    early_stopping = keras.callbacks.EarlyStopping(
        monitor ='val_loss',
        min_delta = 0.001,
        patience = 50, 
        restore_best_weights = True,
        mode = "auto",
        verbose = 1,
        baseline = None)
    
    builder = ModelBuilder(trial, inputs=keras.Inputs((2048,1365,3)),
                          total_classes=6)
    
    model = builder.get_model()
    params = builder.get_params(2)
    run = wandb_block(**params)
    
    model.fit(X_train, y_train, 
              epochs = builder.epoch, 
              batch_size = builder.batch_size, 
              verbose = 0,
              validation_data = (X_test, y_test),
              callbacks = [
                  WandbMetricsLogger(log_freq = builder.sampling_interval),
                  #WandbModelCheckpoint("models/model.keras"),
                  early_stopping
              ]
             )
    run.finish()
    trial_number = params["name"]
    loss, accuracy = model.evaluate(X_test, y_test, verbose = 0)
    return loss